# DC Real Estate Analysis
Isolating qualified sales, where are the condominiumsand residentialwith the cheapest associated cost/sf in the city?Due to market fluctuations in real estate values since emerging from the Great Recession, we should have a data set that focuses on sales within the past five years and ten years, respectively. Also paying attention to which broad areas have the highest turnover.  

In [1]:
#Dependencies
import pandas as pd
import numpy as np
import plotly.plotly as py

#CSV File to load
file_to_load = "./Data_dc-residential-properties/DC_Properties.csv"
#print ('goodpath')

#Read file and store into Pandas DF. Scatter plots with price and 
real_estate_data_pd = pd.read_csv(file_to_load)
real_estate_data_pd.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning:

Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


# Variable Removal

In [2]:
#Clean DF to remove all unrelated variables
isolated_pd = real_estate_data_pd.drop(columns=["Unnamed: 0", "BATHRM", "HF_BATHRM", "HEAT", "AC", "NUM_UNITS", 
                                                "ROOMS", "BEDRM", "AYB", "YR_RMDL", "EYB", "STORIES", "SALE_NUM", 
                                                "BLDG_NUM", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL",
                                                "KITCHENS", "FIREPLACES", "USECODE", "GIS_LAST_MOD_DTTM",
                                                "CMPLX_NUM", "ASSESSMENT_NBHD", "ASSESSMENT_SUBNBHD", 
                                                "CENSUS_TRACT", "CENSUS_BLOCK", "QUADRANT", "X", "Y"])
isolated_pd.head()

,SALEDATE,PRICE,QUALIFIED,GBA,STYLE,STRUCT,LANDAREA,SOURCE,LIVING_GBA,FULLADDRESS,CITY,STATE,ZIPCODE,NATIONALGRID,LATITUDE,LONGITUDE,WARD,SQUARE
0,2003-11-25 00:00:00,1095000.0,Q,2522.0,3 Story,Row Inside,1680,Residential,NaN,1748 SWANN STREET NW,WASHINGTON,DC,20009.0,18S UJ 23061 09289,38.914680,-77.040832,Ward 2,152
1,2000-08-17 00:00:00,NaN,U,2567.0,3 Story,Row Inside,1680,Residential,NaN,1746 SWANN STREET NW,WASHINGTON,DC,20009.0,18S UJ 23067 09289,38.914683,-77.040764,Ward 2,152
2,2016-06-21 00:00:00,2100000.0,Q,2522.0,3 Story,Row Inside,1680,Residential,NaN,1744 SWANN STREET NW,WASHINGTON,DC,20009.0,18S UJ 23074 09289,38.914684,-77.040678,Ward 2,152
3,2006-07-12 00:00:00,1602000.0,Q,2484.0,3 Story,Row Inside,1680,Residential,NaN,1742 SWANN STREET NW,WASHINGTON,DC,20009.0,18S UJ 23078 09288,38.914683,-77.040629,Ward 2,152
4,NaN,NaN,U,5255.0,3 Story,Semi-Detached,2032,Residential,NaN,1804 NEW HAMPSHIRE AVENUE NW,WASHINGTON,DC,20009.0,18S UJ 23188 09253,38.914383,-77.039361,Ward 2,152


# Calculations for Total Sales and Average Cost/SF

In [3]:
#DF for qualified residential properties (n=45,976 <-- differs from final count below?)
dirty_resi_pd = isolated_pd.drop(columns=["LIVING_GBA"])
resi_pd_0 = dirty_resi_pd.drop(dirty_resi_pd[dirty_resi_pd.SOURCE == "Condominium"].index)
resi_pd_1 = resi_pd_0.drop(resi_pd_0[resi_pd_0.QUALIFIED == "U"].index)
#resi_pd_1.shape

#Calulate cost/sf (PRICE/GBA) and append to resi_pd_1. 
resi_pd_1['COST_SF_RESI'] = resi_pd_1['PRICE']/resi_pd_1['GBA']
#resi_pd_1.head()

#Average Cost/sf & count
ave_resi_pd = resi_pd_1.loc[:,["COST_SF_RESI"]]
ave_resi = ave_resi_pd.mean()[0]
count_resi = ave_resi_pd.count()[0]
print (ave_resi)
print (count_resi)

355.8805122620808
45886


In [4]:
#DF for qualified residential properties (n=30,373<-- differs from final count below?)
dirty_condo_pd = isolated_pd.drop(columns=["GBA"])
condo_pd_0 = dirty_condo_pd.drop(dirty_condo_pd[dirty_condo_pd.SOURCE == "Residential"].index)
condo_pd_1 = condo_pd_0.drop(condo_pd_0[condo_pd_0.QUALIFIED == "U"].index)
#condo_pd_1.shape

#Calulate cost/sf (PRICE/LIVING_GBA) and append to condo_pd_1. 
condo_pd_1['COST_SF_CONDO'] = condo_pd_1['PRICE']/condo_pd_1['LIVING_GBA']
#condo_pd_1.head()

#Average Cost/sf & count
ave_condo_pd = condo_pd_1.loc[:,["COST_SF_CONDO"]]
ave_condo = ave_condo_pd.mean()[0]
count_condo = ave_condo_pd.count()[0]
print (ave_condo)
print (count_condo)

482.9001941526933
30347


# Summary Diagram -- Count and Cost/SF

In [6]:
#Summary Diagram
sale_analysis_summary = pd.DataFrame({
    "Total Number of Residential Sales":count_resi,
    "Average Residential Purchase Price":ave_resi,
    "Total Number of Condominium Sales":count_condo,
    "Average Condominium Purchase Price":ave_condo})

ValueError: If using all scalar values, you must pass an index

# Turnover Analysis -- Isolated by Ward & Square
##### Turnover is one of the metrics to measure the growth and overall market of a region. Areas with high turnover usually signify a strong economy and more desirable area to invest. 

In [ ]:
#Concatinate qualified condo & resi dataframes
resi_turn_pd = resi_pd_1.loc[:,["WARD", "SQUARE"]]
condo_turn_pd = condo_pd_1.loc[:,["WARD", "SQUARE"]]
frames = [resi_turn_pd, condo_turn_pd]
turnover_pd = pd.concat(frames)
#turnover_pd.head()

turn_count = turnover_pd.groupby(["WARD"]).count()["SQUARE"]
turn_count

turn = pd.DataFrame({
    "Turover by Ward":turn_count})
turn

# Sale Date Analysis
##### Where have the most sales occurred within the last five and ten years? Where have prices increased the most and least? Since we do not have addresses on the condos, we can use Wards and/or Zip Codes. There are eight Wards and 65 zip codes. We could fine tune it by using the binning and group method, however, that may be fairly time consuming. 

# What is still needed?
> * Statistical analysis of Mina's choosing
> * Sales Analysis for last five and ten years
> * Any other ideas for what you may want to look at?
> * Graphs
    * Scatter plot of all quaified sales
    * Graphical representation of ward turnover. Histogram or map overlay? Matt can build map in InDesign if that is easier and faster. 
    * Histogram of sale dates
    * There are probably some other graphical ideas that you all could come up with. You will see that a number of variables were removed from the original data set to clean it up. You are more that welcome to bring any of those in. 
> * Begin to rough out the presentation. 